# Image Classification in the Browser

In this notebook, we will show how to create a `.air` file to perform image classification in the browser using a neural network.  To do this, we will utilize the CIFAR-10 dataset to build the initial model, prune the model using the `beyondml` package, and then package the model using the `aisquared` Python SDK.

## Dependencies

For this notebook, the following dependencies are required:

- `beyondml`
- `aisquared`

Both of these are available on [pypi](https://pypi.org) via `pip`.  The following cell also runs the commands to install these dependencies as well as imports them into the notebook environment, along with TensorFlow (which is a dependency of the `beyondml` package).

In [ ]:
! pip install beyondml
! pip install aisquared

from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import aisquared
import beyondml.tflow as mann

## Model Creation

Now that the required packages have been installed and imported, it is time to create the sentiment analysis model.  To do this, we have to first download and preprocess the data, create the model, prune the model so that it can perform well in the browser, and then package the model in the `.air` format.  The following cells will go through an in-depth explanation of each of the steps in this process.

In [ ]:
# Loading the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = tf.image.resize(x_train, [30, 30])/255
x_test = tf.image.resize(x_test, [30, 30])/255

label_map = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]

In [ ]:
# Create the model

input_layer = tf.keras.layers.Input(x_train.shape[1:])
x = mann.layers.MaskedConv2D(
    16,
    padding = 'same',
    activation = 'relu'
)(input_layer)
x = mann.layers.MaskedConv2D(
    16,
    padding = 'same',
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D(
    2,
    strides = 1,
    padding = 'valid'
)(x)
x = mann.layers.MaskedConv2D(
    32,
    padding = 'same',
    activation = 'relu'
)(x)
x = mann.layers.MaskedConv2D(
    32,
    padding = 'same',
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D(
    2,
    strides = 1,
    padding = 'valid'
)(x)
x = mann.layers.MaskedConv2D(
    64,
    activation = 'relu'
)(x)
x = mann.layers.MaskedConv2D(
    64,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D(
    2,
    strides = 1,
    padding = 'valid'
)(x)
x = tf.keras.layers.Flatten()(x)
x = mann.layers.MaskedDense(256, activation = 'relu')(x)
x = mann.layers.MaskedDense(256, activation = 'relu')(x)
output_layer = mann.layers.MaskedDense(10, activation = 'softmax')(x)

model = tf.keras.models.Model(input_layer, output_layer)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
# Perform initial sparsification
model = mann.utils.mask_model(
    model,
    20,
    x = x_train[:1000],
    y = y_train[:1000]
)

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

# Create a pruning callback that will increase pruning rate as performance improves
callback = mann.utils.ActiveSparsification(
    performance_cutoff = 0.60,
    starting_sparsification = 20,
    max_sparsification = 80,
    sparsification_rate = 5
)

# Train the model with the sparsification callback
model.fit(
    x_train,
    y_train,
    epochs = 1000,
    batch_size = 512,
    validation_split = 0.2,
    verbose = 2,
    callbacks = [callback]
)

# Now that the model has been trained, convert all model layers to built-in TensorFlow layers
model = mann.utils.remove_layer_masks(model)

In [ ]:
# Check model performance
preds = model.predict(x_test).argmax(axis = 1)
print('Model Performance on Test Data:')
print('\n')
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))

# Save the model
model.save('ImageClassifier.h5')

## Package the Model

Now that the model has been created, we can package the model into a single `.air` file that enables integration into the browser.

To perform this packaging, we will be utilizing the `aisquared` package.

In [ ]:
harvester = aisquared.config.harvesting.ImageHarvester()
preprocesser = aisquared.config.preprocessing.image.ImagePreprocessor(
    [
        aisquared.config.preprocessing.image.Resize([30, 30]),
        aisquared.confgi.preproccessing.image.DivideValue(255)
    ]
)
analytic = aisquared.config.analytic.LocalModel('ImageClassifier.h5', 'cv')
postprocesser = aisquared.config.postprocessing.MulticlassClassification(label_map, include_probability = True)
renderer = aisquared.config.rendering.ImageRendering()

config = aisquared.config.ModelConfiguration(
    'cifar10classifier',
    harvester,
    preprocesser,
    analytic,
    postprocesser,
    renderer
)